In [4]:
import numpy as np
import pandas as pd
import pylab as pl
import os

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi
import json
  
with open('kaggle.json') as f:
    data = json.load(f)

os.environ['KAGGLE_USERNAME'] = data["username"]
os.environ['KAGGLE_KEY'] = data["key"]

api = KaggleApi()
api.authenticate()

In [6]:
api.dataset_download_files('rmisra/news-category-dataset', path="./data", unzip=True)

In [7]:
data = pd.read_json("data/News_Category_Dataset_v2.json", lines=True)

In [8]:
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [31]:
data["full_text"] = data["headline"] + " " + data["short_description"]

In [32]:
X, y = data["full_text"].to_numpy(), data["category"].to_numpy()

In [33]:
# Encode y labels
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

In [34]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[:1000], y[:1000], test_size=0.2)

In [35]:
# Tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=True, min_df=0.001, max_df=0.7, sublinear_tf=True, strip_accents="unicode", ngram_range=(1,2))

In [36]:
# Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(lowercase=True, min_df=0.0001, max_df=0.7, strip_accents="unicode", ngram_range=(1,1), stop_words="english")

In [37]:
# logreg
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=4, max_iter=1000, solver="saga", verbose=1, tol=0.1)

In [38]:
# naive bayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [41]:
# Pipeline
from sklearn.pipeline import Pipeline
pipe = Pipeline([('vectorizer', count_vectorizer), ('model', logreg)])
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

Epoch 1, change: 1.00000000
Epoch 2, change: 0.29137305
Epoch 3, change: 0.14885496
Epoch 4, change: 0.10644563
convergence after 5 epochs took 0 seconds


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    0.0s finished


0.665